# Polytope requests examples - High Resolution Surface forecast data

This notebook holds example request for getting Surface forecast data from ECMWF's High Resolution forecast. It is the first in set of notebooks to help you get ECMWF data using Polytope API.   

Before we can run the requests we need to install and import polytope client (and xarray to later inspect the data) and start the client.

In [ ]:
!python3 -m pip install --upgrade git+https://git.ecmwf.int/scm/lex/polytope-client.git@master

If you're working on **mybinder** you should already have **xarray, matplotlib** and **cfgrib** installed.  
If you're working locally and don't have them installed, please run the next cell.

In [ ]:
import sys
!conda install --yes --prefix {sys.prefix} xarray cfgrib matplotlib

In [ ]:
from polytope.api import Client

In [ ]:
import xarray as xr

Insert here your email and ecmwf token:

In [ ]:
c = Client(user_email = 'johndoe@ecmwf.int',
           user_key = '4j3s3d34n4sn335jacf3n3d4f4g61635')

We can list all the collections. Different users will have the access to different set of collections. HiDALGO users have the access to **hidalgo-mars** collection.

In [ ]:
c.list_collections()

### Get some surface parameters
This example request will download a couple of surface forecast parameters. 

Most of the high resolution data is in stream **oper**. Type **fc** will give us forecast data. Since it is one surface data, we don't need to use keyword 'levelist'.  
Full list of available SFC parameters you can find in archive catalogue on this link:  
https://apps.ecmwf.int/archive-catalogue/?stream=oper&levtype=sfc&time=00%3A00%3A00&expver=1&month=mar&year=2020&date=2020-03-08&type=fc&class=od  
ECMWF High resolution forecast is available at following steps:  
1 hourly from 0 to 90  
3 hourly from 93 to 144 and  
6 hourly from 152 to 240 (360)  

This request will download forecast data with starts on 5th March 2020 at 00 and 12 UTC, and steps from 0 to 11.  
Parameters that will be downloaded are 2t - 2 metre temperature, and 10u/10v - u and v component of wind at 10 m.  
**EXPVER** is version of forecasting model. Always keep this as 0001 as that is the latest model version.  

**AREA** is geographical area. Numbers correspond to **North/West/South/East**.  

**GRID** is controlling horizontal resolution of the data. High resolution data is in 0.1x0.1 and we can download any multiply of this reslution (0.2x0.2, 0.3x0.3....)

In [ ]:
request = {
    'stream': 'oper',
    'levtype': 'sfc',
    'param': '2t/10u/10v',
    'step': '0/1/2/3/4/5/6/7/8/9/10/11',
    'time': '00/12',
    'date': '20201105',
    'type': 'fc',
    'class': 'od',
    'expver': '0001',
    'area': '50/15/40/25',
    'grid': '0.5/0.5'
}

To retrieve the data just run the retrieve.

In [ ]:
c.retrieve('hidalgo-mars', request, 'sfc.grib')

In [ ]:
ds = xr.open_dataset('sfc.grib',engine='cfgrib')
ds

You can also append to an exsisting file. We will append the data from next day to the already dowloaded data.

In [ ]:
request2 = {
    'stream': 'oper',
    'levtype': 'sfc',
    'param': '2t/10u/10v',
    'step': '0/1/2/3/4/5/6/7/8/9/10/11',
    'time': '00/12',
    'date': '20201106',
    'type': 'fc',
    'class': 'od',
    'expver': '0001',
    'area': '50/15/40/25',
    'grid': '0.5/0.5'
}

In [ ]:
# Append to an existing file
c.retrieve('hidalgo-mars', request2, 'sfc.grib', append = True)


If we inspect data now, we can notice that there are 4 start times instead of 2.

In [ ]:
ds = xr.open_dataset('sfc.grib',engine='cfgrib')
ds

We can plot just one field to see if the data we got makes sense

In [ ]:
ds.t2m.sel(time='2020-11-05T00:00:00.000000000',step='01:00:00').plot()

We can list our active requests:

In [ ]:
ids = c.list_requests()

We can get the data from different dates in one request too.

In [ ]:
request3 = {
    'stream': 'oper',
    'levtype': 'sfc',
    'param': '2t/10u/10v',
    'step': '0/1/2/3/4/5/6/7/8/9/10/11',
    'time': '00/12',
    'date': '20200306/20200307/20200308/20200309/20200310',
    'type': 'fc',
    'class': 'od',
    'expver': '0001',
    'area': '50/15/40/25', #NWSE
    'grid': '0.5/0.5'
}

In [ ]:
c.retrieve('hidalgo-mars', request3, 'sfc_3.grib')

In [ ]:
ds = xr.open_dataset('sfc_3.grib',engine='cfgrib')
ds

If we need several consecutive days we can write something like this and get the data from 6 to 10 March 2020.

In [ ]:
request4 = {
    'stream': 'oper',
    'levtype': 'sfc',
    'param': '2t/10u/10v',
    'step': '0/1/2/3/4/5/6/7/8/9/10/11',
    'time': '00/12',
    'date': '20200306/to/20200310',
    'type': 'fc',
    'class': 'od',
    'expver': '0001',
    'area': '50/15/40/25',
    'grid': '1/1'
}

In [ ]:
c.retrieve('hidalgo-mars', request4, 'sfc_4.grib')

In [ ]:
ds = xr.open_dataset('sfc_4.grib',engine='cfgrib')
ds

### Submitting a list of requests
Sometimes we want to split our request into 2 smaller requests. We can then submit both requests together as a list.  
Let's make a request that accesses the same set of data from the same date, just run from 12 UTC.

In [ ]:
request_A = {
    'stream': 'oper',
    'levtype': 'sfc',
    'param': '2t/10u/10v',
    'step': '0/1/2/3/4/5/6/7/8/9/10/11',
    'time': '00',
    'date': '20200306',
    'type': 'fc',
    'class': 'od',
    'expver': '0001',
    'area': '50/15/40/25',
    'grid': '1/1'
}
request_B = {
    'stream': 'oper',
    'levtype': 'sfc',
    'param': '2t/10u/10v',
    'step': '0/1/2/3/4/5/6/7/8/9/10/11',
    'time': '12',
    'date': '20200306',
    'type': 'fc',
    'class': 'od',
    'expver': '0001',
    'area': '50/15/40/25',
    'grid': '1/1'
}

In [ ]:
c.retrieve('hidalgo-mars', [request_A,request_B], 'sfc_AB.grib')

In [ ]:
ds = xr.open_dataset('sfc_AB.grib',engine='cfgrib')
ds

### Asynchronous retrieval 
If we want to submit a request but run it in background so we can do other things, asynchronous retrieval can also be done. 

In [ ]:
r = c.retrieve('hidalgo-mars', [request_A,request_A], 'asyn_AB.grib',
               asynchronous = True)

Now we can see the info of our requests or download the data when it is ready.  
Each request is a member of the list so we can access each of them individually (`r[0]` or `r[1]` in our case).

In [ ]:
r[1].describe()

And download.  
You can only dowlnoad each request from the list separately. By default they will be appended in the same file.  If for any reason you don't want this you can set parameter append=Flase.

In [ ]:
r[0].download()

In [ ]:
r[1].download()

We can inspect what is in our grib using xarray. 

In [ ]:
ds = xr.open_dataset('asyn_AB.grib',engine='cfgrib')
ds